In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics
from sklearn import tree
from sklearn.metrics import classification_report, accuracy_score
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow import keras
from keras.api._v2.keras import activations
from sklearn.preprocessing import StandardScaler

In [4]:
kidney_df=pd.read_csv('chronickidney_disease.csv')


In [5]:
d=kidney_df.duplicated().sum()

In [6]:
n=kidney_df.isnull().sum()

In [7]:
kidney_df=kidney_df.drop(['id'], axis=1)

In [8]:
encode = LabelEncoder()
for col in kidney_df.columns:
    if kidney_df[col].dtype == 'O':  # convert string columns to be able to apply feature selection & models
        kidney_df[col] = encode.fit_transform(kidney_df[col])
    #kidney_df[col].fillna(kidney_df[col].mode()[0], inplace=True)

In [9]:
kidney_df = kidney_df.fillna(method='bfill')

In [10]:
kidney_df

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,2,1,0,0,121.0,...,32,72,34,1,4,1,0,0,0,0
1,7.0,50.0,1.020,4.0,0.0,2,1,0,0,423.0,...,26,56,49,0,3,1,0,0,0,0
2,62.0,80.0,1.010,2.0,3.0,1,1,0,0,423.0,...,19,70,49,0,4,1,1,0,1,0
3,48.0,70.0,1.005,4.0,0.0,1,0,1,0,117.0,...,20,62,19,1,3,1,1,1,1,0
4,51.0,80.0,1.010,2.0,0.0,1,1,0,0,106.0,...,23,68,27,0,3,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,1,1,0,0,140.0,...,35,62,30,0,3,1,0,0,0,2
396,42.0,70.0,1.025,0.0,0.0,1,1,0,0,75.0,...,42,72,44,0,3,1,0,0,0,2
397,12.0,80.0,1.020,0.0,0.0,1,1,0,0,100.0,...,37,61,36,0,3,1,0,0,0,2
398,17.0,60.0,1.025,0.0,0.0,1,1,0,0,114.0,...,39,67,41,0,3,1,0,0,0,2


In [11]:
kidney_df.to_csv('updated_kideny.csv', index=False)

In [12]:
X = kidney_df.loc[:, kidney_df.columns != col]
Y = kidney_df['classification']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1,shuffle=True)

In [13]:
X_train.shape


(320, 24)

In [14]:
X.shape


(400, 24)

In [15]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
svm_clf =svm.SVC(kernel='rbf',gamma='auto',C=2)
svm_clf.fit(X_train, Y_train)
PREDICTION=svm_clf.predict(X_test)
PREDICTION

array([2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0,
       2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0,
       0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0])

In [17]:
r=sklearn.metrics.confusion_matrix(Y_test, PREDICTION)
r=np.flip(r)
acc1=(r[0][0]+r[-1][-1])/np.sum(r)
acc1

1.0

In [18]:
logistic_clf = LogisticRegression()
logistic_clf.fit(X_train, Y_train)
y_pred = logistic_clf.predict(X_test)
acc2= accuracy_score(Y_test, y_pred)
acc2

0.9875

In [19]:
y_pred

array([2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0,
       2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0,
       2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0,
       0, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0])

In [20]:
coef_num= logistic_clf.coef_

In [21]:
coef_num

array([[-3.24989504e-01,  4.94098360e-01, -8.79477985e-01,
         5.10455312e-01,  4.32466704e-01,  3.21970694e-01,
        -3.81354691e-01, -3.48586586e-01,  1.67883595e-01,
         3.51106290e-01,  2.84236053e-01,  6.28202629e-01,
        -2.62854149e-01, -6.51739117e-02, -1.21310874e+00,
        -2.70392753e-01, -2.47286331e-01,  4.83554203e-01,
         2.13772573e-01, -2.57239974e-02, -2.41051167e-03,
        -1.28576567e-01,  4.99107267e-01, -2.73625771e-01],
       [ 3.37058797e-01, -3.37703646e-01, -2.42458383e-01,
         7.64951345e-01, -2.42422555e-01,  2.83173888e-01,
         8.06929821e-02,  4.88392861e-01, -2.20278830e-01,
         5.96320682e-03, -3.50973241e-01, -9.21657219e-04,
        -9.61736044e-02,  5.10929198e-02, -3.03102160e-01,
         2.96268361e-01,  2.63685197e-01, -1.38267412e-01,
         5.70513805e-01,  5.04211683e-01, -3.53958562e-01,
         8.72431630e-01, -5.46435455e-01,  1.51258970e-01],
       [-1.20692926e-02, -1.56394714e-01,  1.12193637e

In [22]:
intercipt= logistic_clf.intercept_

In [23]:
intercipt

array([ 3.66784946, -2.93963134, -0.72821812])

In [24]:
kn_clf=KNeighborsClassifier(n_neighbors=1)
kn_clf.fit(X_train,Y_train)
y1= kn_clf.predict(X_test)
acc3=accuracy_score(Y_test,y1)
acc3

0.9625

In [25]:
dtree_clf= tree.DecisionTreeClassifier()
dtree_clf.fit(X_train, Y_train)
y2= dtree_clf.predict(X_test)
acc4=accuracy_score(Y_test,y2)
acc4

1.0

In [26]:
model= keras.Sequential()
hidden_layer_sizes= coef_num.shape[1:]
model.add(keras.layers.Dense(units=hidden_layer_sizes[0],activation='relu',input_shape=(X_train.shape[1],)))
for layer_size in hidden_layer_sizes[1:]:
  model.add(keras.layers.Dense(units=layer_size,activation='relu'))

model.add(keras.layers.Dense(units=coef_num[-1].shape[0],activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                600       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
Total params: 1,200
Trainable params: 1,200
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train , Y_train, epochs=200, batch_size=173)


Epoch 1/200
2/2 [==============================] - 1s 10ms/step - loss: 0.3340 - accuracy: 0.9594
Epoch 2/200
2/2 [==============================] - 0s 7ms/step - loss: 0.3207 - accuracy: 0.9594
Epoch 3/200
2/2 [==============================] - 0s 9ms/step - loss: 0.3078 - accuracy: 0.9594
Epoch 4/200
2/2 [==============================] - 0s 8ms/step - loss: 0.2955 - accuracy: 0.9594
Epoch 5/200
2/2 [==============================] - 0s 8ms/step - loss: 0.2837 - accuracy: 0.9656
Epoch 6/200
2/2 [==============================] - 0s 7ms/step - loss: 0.2728 - accuracy: 0.9656
Epoch 7/200
2/2 [==============================] - 0s 8ms/step - loss: 0.2624 - accuracy: 0.9656
Epoch 8/200
2/2 [==============================] - 0s 12ms/step - loss: 0.2520 - accuracy: 0.9656
Epoch 9/200
2/2 [==============================] - 0s 9ms/step - loss: 0.2427 - accuracy: 0.9656
Epoch 10/200
2/2 [==============================] - 0s 9ms/step - loss: 0.2336 - accuracy: 0.9688
Epoch 11/200
2/2 [=========

In [29]:
# input >> 48,80,1.02,1,0,2,1,0,0,121,36,1.2,111,2.5,15.4,32,72,34,1,4,1,0,0,0
print("Enter Values of Kidney:")
list = [[float(input()) for j in range(24)]]
list = np.array(list)
prediction = model.predict(list)
prediction = prediction.all()
if prediction == 0:
  prediction = 0 #chronic
else:
  prediction = 1 #non_chronic
print(prediction)

Enter Values of Kidney:
48
80
1.02
1
0
2
1
0
0
121
36
1.2
111
2.5
15.4
32
72
34
1
4
1
0
0
0
1/1 [==============================] - 0s 76ms/step
0


In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
##converter.experimental_new_converter = True

tflite_model = converter.convert()

with open('model.tflite_kidney', 'wb') as f:
  f.write(tflite_model)

